# Julia nivel intermedio

Hemos visto varias cosas:

- variables, incluyendo diversos tipos numéricos(`Int64`, `Float64`, `BigFloat`, `Complex{Float64}`, `Rational{Int64}`), cadenas, vectores, matrices

- ciclos (`for`...`end`) y rangos (`a:b`)

- diversas formas de condicionales

- funciones simples

Seguiremos con esto, pero de una manera más sistemática enfatizando conceptos *julianos*.

Una pieza básica del curso se puede resumir en:

> ## Learning by doing

Piénsenlo y, sobre todo, ¡practíquenlo!

## Ejemplo: el método de Newton

Como bien sabemos, el método de Newton es un método iterativo para encontrar los ceros, o raíces, de una ecuación $f(x)=0$. A partir de una aproximación $x_0$, y denotando la derivada de $f(x)$ como $f'(x)$, tenemos:

\begin{equation}
x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)} .
\end{equation}



Para ejemplificar las cosas usaremos la función $f(x) = x^2-2$:

In [ ]:
f(x) = x^2 - 2.0

... y su derivada:

In [ ]:
f´(x) = 2*x

Entonces, el método de Newton vendría dado por la función:

In [ ]:
x_0 = 3.0

for i in 1:10
    x_new = x_0 - f(x_0) / f´(x_0)
    println(i, "\t", x_new)
    x_0 = x_new
end

In [ ]:
sqrt(2.0)

Recordando la clase de cálculo I, el método de Newton sólo converge si $x_0$ es *suficientemente* cercano a la raíz. Veamos qué pasa si *variamos* la condición inicial $x_0$.

In [ ]:
initial_guess = -3:0.125:3

In [ ]:
typeof(initial_guess)

In [ ]:
collect(initial_guess)   # use tab-completion <TAB> for long variable names!

Defino el arreglo de salida que, entrada por entrada, corresponderá a las condiciones iniciales definidas en `initial_conditions`.

In [ ]:
roots = similar(initial_guess)

El número de datos de `initial_guess` es precisamente 49:

In [ ]:
length(initial_guess)

Ahora, implementamos todo junto; abajo, `enumerate(initial_conditions)` regresa una *tupla* con un enetero, que es el índice de la condición inicial (`j`), y su valor (`x_0`)

In [ ]:
for (j, x_0) in enumerate(initial_guess)
    x = x_0

    # 100 iterates of Newton's method
    for i in 1:100
        x = x - f(x) / f´(x)
    end
    
    roots[j] = x
end

Veamos los resultados:

In [ ]:
showall(roots)

**Pregunta:** ¿Por qué hay un `NaN`?

Ahora vamos a visualizar esto. Para eso usaremos el paquete "PyPlot", que es una interface para usar "matplotlb".

Para instalarlo es necesario usar el siguiente comando:
```julia
    Pkg.add("PyPlot")
```

Otros paquetes *registrados* se pueden encontrar [aquí](http://pkg.julialang.org/).

Para empezar a usar un paquete, en este caso "PyPlot" usamos el comando:

```julia
    using PyPlot
```

(Puede tardar un poco la primera vez que se utiliza.)

In [ ]:
using PyPlot

In [ ]:
figure(figsize=(6,4)) # este comando define el tamaño de la figura
plot(roots)  # este pinta las componentes "y" (`roots`); en "x" se utiliza el índice del vector

## Rendimiento

Ahora, algo **muy** importante en julia, es que el alto rendimiento se da cuando uno hace el trabajo dentro de funciones, y **no** en el *global scope*.

Así que lo anterior, lo empacamos dentro de una función.

In [ ]:
"""
This function computes the roots of `f(x)` (which must be defined before) for different initial 
iterates in the range [-20,20], and returns them packed in a vector.
"""
function compute_roots()
    initial_conditions = -20:0.125:20
    roots = similar(initial_conditions)

    for (j, x_0) in enumerate(initial_conditions)
        x = x_0

        # 100 iterations of Newton's method
        for i in 1:100
            x = x - f(x) / f´(x)
        end

        roots[j] = x
    end
    
    roots
end

Noten la salida de la definición de la función `compute_roots`:

```
    compute_roots (generic function with 1 method)
```

A esto volveremos más adelante...

In [ ]:
?compute_roots

In [ ]:
# ";" quita la salida
roots = compute_roots();

¿Cuánto tiempo tardó esto?

El *macro* `@time` permite precisamente medir el tiempo de ejecución.

Julia *compila* cada función la primera vez que es utilizada. Es por eso que conviene compilar `@time`, y luego la ejecución de `@time` con la función que nos interesa medir. Así, para medir fehacientemente el tiempo de ejecución, es importante no incluir en esa medición la compilación de la función.

In [ ]:
@time 1;

In [ ]:
@time 1;

Las dos ejecuciones anteriores muestran que, al compilar (primer ejecución), julia utiliza espacio de memoria; una vez compiladas las cosas, el uso de memoria disminuye de manera importante. Incidentalmente, el uso exagerado de memoria muestra que las cosas pueden ser hechas de mejor manera...

In [ ]:
@time compute_roots();

In [ ]:
@time compute_roots();

**Ejercicio 4:** ¿Qué conviene más, usar un ciclo `for... end` o un ciclo `while... end` en la función `compute_roots()`, respecto al for más interno?

## Funciones genéricas

Lo que hemos hecho hasta ahora está bien, pero *no* es muy genérico, en el sentido de que no es muy cómodo la implementación si queremos considerar distintas funciones `f(x)`.

Para esto, *definimos* el siguiente *método* para la función `compute_roots`:

In [ ]:
function compute_roots(f, f´)
    initial_conditions = -20:0.125:20
    roots = similar(initial_conditions)

    for (j, x_0) in enumerate(initial_conditions)
        x = x_0

        # 100 iterations of Newton's method
        for i in 1:100
            x = x - f(x) / f´(x)
        end

        roots[j] = x
    end
    
    roots
end

Es importante notar que, ahora, tenemos 2 métodos definidos para la *misma* función `compute_roots`. Esto es, **no** hemos sobreescrito la función `compute_roots`, sino que hemos creado un nuevo método. 

Los métodos se distinguen por el tipo de argumentos de la función: en un caso *no* hay ningún argumento, mientras que en el otro hay dos argumentos.

La instrucción `methods` ayuda a saber cuántos métodos hay asociados a una función, incluyendo información sobre el tipo de los parámetros; esto último es la "signatura de tipo" (*type signature*).

In [ ]:
methods(compute_roots)

**Ejercicio 5:** ¿De qué tipo son las variables f y f´?

Ejecutamos ahora la nueva implementación de las funciones:

In [ ]:
@time compute_roots(f, f´);

In [ ]:
@time compute_roots(f, f´);

Vale la pena notar que, la vieja implementación es más rápida que la nueva, incluyendo un mejor uso de la memoria. Esto se debe a que julia *no* sabe cómo inferir el resultado de $f$ y $f'$. 

La manera de notar que `compute_roots(f,f´)` va a dar problemas (sobreuso de memoria) se obtiene usando el macro `@code_warntype`; este macro da información sobre la inferencia de tipos que puede hacer el compilador. En las siguientes dos ejecuciones, vale la pena *notar* en el apartado de variables que la variable `x` en un caso es un `Complex{Float64}` y en el otro es `Any`, respectivamente para `compute_roots()` y `compute_roots(f,f´)`; obtener `Any` es *mala señal*:

In [ ]:
@code_warntype(compute_roots())

In [ ]:
@code_warntype(compute_roots(f, f´))

En Julia hay otro manera de definir y usar funciones: las funciones anónimas. Por ejemplo:

In [ ]:
@time compute_roots(x->x^2-2, x->2x);

In [ ]:
@time compute_roots(x->x^2-2, x->2x);

En este momento (versión 0.4 de julia), las funciones anónimas son aún lentas, pero se pueden hacer trucos para que funcionen mejor. Nuevamente, esto tiene que ver con el hecho de que julia no sabe el tipo de resultado de las funciones anónimas, en el momento de compilar la función.

Hagamos otro ejemplo un poco más interesante:

In [ ]:
@time roots = compute_roots( x -> (x-1)*(x-2)*(x-3), x->3x^2-12x+11);

In [ ]:
@time roots = compute_roots( x -> (x-1)*(x-2)*(x-3), x->3x^2-12x+11);

In [ ]:
figure(figsize=(6,4))
plot(-20:0.125:20, roots)
ylim(0,4)

In [ ]:
figure(figsize=(6,4))
plot(-20:0.125:20, roots, "g.-")
ylim(0,4)
xlim(1,3)


## El método de Newton sobre los complejos

Ahora, implementaremos el método de Newton, para alguna función modelo ($f(z) = z^3-1$), pero usaremos condiciones iniciales en los complejos.

Las condiciones iniciales, igual que antes, las definiremos a partir de un `FloatRange{Float64}`, que usaremos tanto para la parte real como para la parte imaginaria de $z_0$.

A priori podríamos proceder como antes. Sin embargo, hay *sutilezas*, ya que la salida `roots` no será un vector, sino una matriz.

In [ ]:
f(z) = z^3-one(z)
f´(z) = 3*z^2

**NOTA**: La siguiente función necesita las funciones $f$ y $f'$, lo que permite usarlas en contextos más generales. Esto, como vimos antes, tiene una penalización en la ejecución.

In [ ]:
function compute_complex_roots(f, f´, range=-5.0:0.125:5.0)  # default value

    L = length(range)
    
    #### VER EL EJERCICIO 6: Dos opciones para *declarar* `roots`:
    ## Uno: definir la matriz, y tener todos los elementos inicialmente en cero
    roots = zeros(Complex128, L, L)  # set a matrix of appropriate length to zero
    ## Dos: definir *el tipo y dimensiones* de la matriz
    # roots = Array(Complex128, L, L)

    for (j, x) in enumerate(range)
        for (i, y) in enumerate(range)
            
            z = x + y*im
            
            for k in 1:1000
                z = z - f(z) / f´(z)
            end
            
            roots[i,j] = z
            
        end
    end
    
    roots
end

In [ ]:
@time compute_complex_roots(f, f´, -5.0:1.0:5.0);

In [ ]:
@time croots = compute_complex_roots(f, f´, -5:0.01:5);

**Ejercicio 6:** Encuentren la implementación más rápida de `compute_complex_roots` considerando:
- Las dos opciones para definir inicialmente `roots`
- Los índices (i,j) que aparecen en el doble ciclo, respecto a su orden en `roots`

Entre las posibles implementaciones, ¿cuál es la diferencia en el tiempo (o el cociente) entre la implementación más rápida y la más lenta?

¿Por qué `compute_complex_roots` tiene 2 métodos definidos?

Visualicemos los resultados: para esto usaremos `imshow`, que sirve para visualizar una matriz, y el código de colores lo definiremos a partir de la parte imaginaria de `roots`.

In [ ]:
imshow(imag(croots))

Juguemos un poco con esto...

In [ ]:
imshow(imag(compute_complex_roots(f, f´, -0.3:0.001:0.7)))

## Definición de nuevos tipos

Algo que definitivamente hasta ahora **no** hemos visto es cómo crear nuevos tipos. Esto es algo impresionantemente útil, ya que permite un arreglo compacto (y tal vez eficiente) de los datos que, si es así, se traduce en velocidad.

Como ilustración definiremos un tipo nuevo, `Vector2D`, que permite dar la posición, o velocidad, de una partícula *libre*.

### Los campos de un `type`

Para entender un poco más lo que significa definir un nuevo tipo, consideremos los números complejos:

In [ ]:
z0 = complex(3,2) # Esto es *mejor* que z=3+2im

In [ ]:
typeof(ans)

La instrucción `fieldnames` da los campos internos de un tipo; vale la pena notar que el resultado consiste en un vector de *símbolos*. Estos símbolos son los nombres de los campos internos del tipo.

In [ ]:
fieldnames(z0)

In [ ]:
z0.re, z0.im

In [ ]:
typeof(ans)

### Vectores de tamaño fijo

La sintáxis básica para definir nuevos tipos es:

In [ ]:
type Vector2D
    x :: Float64
    y :: Float64
end

En la celda anterior, el doble dos-puntos, `::`, sirve para *anotar* el tipo de las variables internas. Esto es, `Vector2D` constará de dos componentes, `x` y `y`, ambas del tipo `Float64`.

Por **convención** es que las estructuras de tipo empiezan con una *mayúscula*; las funciones empiezan en *minúsculas*.

A estas alturas del partido, sólo hemos definido el tipo `Vector2D`; ¿hemos definido algún método asociado a ésto? O, en otras palabras, ¿cómo creamos algo que sea del tipo `Vector2D`?

In [ ]:
methods(Vector2D)

Usando el resultado de `methods`, probamos con la función `call`:

In [ ]:
call(Vector2D, 1.0, 2.0)

In [ ]:
call(Vector2D, 1, 2)

In [ ]:
call(Vector2D, (1, 2)...) # "..." es el 'splat' operator

La función `call` de hecho es la forma genérica de crear cualquier tipo; uno puede usar la forma *concreta*:

In [ ]:
x = Vector2D(1, 4)

In [ ]:
y = Vector2D(5, 6)

In [ ]:
fieldnames(Vector2D)

In [ ]:
x.x

Vale la pena notar que uno puede modificar los campos internos de un tipo:

In [ ]:
x.x = 3

In [ ]:
x

Pensando que estas cantidades son vectores desplazamiento, o velocidades, quisiéramos sumarlos. Problemos que obtenemos...

In [ ]:
x + y

Julia nos da un error ya que *no* está definida la función `+` para los argumentos de tipo `Vector2D`; el mensaje de la primer línea es perfectamente claro.

Entonces, cómo *no* está definido, y nos interesa usarlo, podemos simplemente definirlo.

In [ ]:
+(a::Vector2D, b::Vector2D) = Vector2D(a.x+b.x, a.y+b.y)

(En un momento explicaremos el mensajo y cómo hacer las cosas bien.)

In [ ]:
x + y

¿Y si queremos definir el producto interno, o sea, la función `dot`?

In [ ]:
x ⋅ y # x \cdot<TAB> y

Entonces, tratamos de proceder de igualmanera

In [ ]:
dot(a::Vector2D, b::Vector2D) = a.x*b.x + a.y*b.y

In [ ]:
x ⋅ y

In [ ]:
methods(Base.dot)

Julia simplemente **no** nos permite si quiera definir esta función.

La manera de proceder aquí es, extendiendo los métodos de `dot` al caso que nos interesa. Sin embargo, debemos *explícitamente* importar la función que será extendida. 

Una manera de hacer esto es:

In [ ]:
Base.dot(a::Vector2D, b::Vector2D) = a.x*b.x + a.y*b.y

In [ ]:
x ⋅ y

Esto está relacionado con el mensaje que apareció al definir `+(a::Vector2D, b::Vector2D)`. Lo correcto hubiera sido definirla usando:

```julia
Base.+(a::Vector2D, b::Vector2D)
```

Una manera alternativa, que a veces es más conveniente, es usando el comando

```julia
import Base.dot
```

cosa que debe ejecutarse *antes* de definir `dot`. Para ilustrar esto debemos *limpiar* el espacio de nombres en la sesión actual. Esto lo hacemos con `workspace()`, lo que casi equivale a resetear la sesión.

In [ ]:
workspace()

In [ ]:
x

In [ ]:
dot

In [ ]:
Vector2D

In [ ]:
type Vector2D
    x :: Float64
    y :: Float64
end

In [ ]:
import Base: +, dot

In [ ]:
+(a::Vector2D, b::Vector2D) = Vector2D(a.x+b.x, a.y+b.y)

In [ ]:
dot(a::Vector2D, b::Vector2D) = a.x*b.x + a.y+b.y

In [ ]:
methods(dot)

In [ ]:
x = Vector2D(3, 4)
y = Vector2D(5, 6)

Para saber qué método utilizó cierta ejecución de algún código, por ejemplo, `Vector2D(3, 4)`, utilizo el macro `@which`:

In [ ]:
@which Vector2D(3, 4)

In [ ]:
x + y

In [ ]:
x ⋅ y

### Tipos parametrizados

Hay situaciones en que nos interesa definir un nuevo tipo para distintos tipos de signatura de los parámetros internos. Un ejemplo es `Complex` o `Rational`, pero de hecho también los arreglos son tipos que reconocen la estructura de los elementos internos que los componen:

In [ ]:
typeof(complex(1.0,2.0))

In [ ]:
typeof(complex(1,2))

In [ ]:
typeof([1, 2.1])

Noten arriba que el entero "1" es *promovido* a Float64 "1.0", que es el tipo del segundo elemento.

In [ ]:
workspace()

In [ ]:
immutable Vector2D{T}
    x::T
    y::T
end

Arriba, hemos usado `immutable` en lugar de `type`, lo que le permite al compilador guardar (en memoria) a este tipo de estructura de una manera más eficiente. Esto, sin embargo, tiene consecuencias, como es el hecho de que una vez definida una estructura, sus componentes *no* pueden cambiar.

En la definición de `Vector2D`, `T` representa el tipo de la estructura que creamos y *parametriza* a `Vector2D` lo que se indica con `{T}`. Dada esta definición, ambas componentes `x` y `y` son del (mismo) tipo `T`.

In [ ]:
x = Vector2D(1,3)

y = Vector2D(2.0,3.5)

println(x)

In [ ]:
x.x = 4

El error se debe a que la estructura es inmutable, y que el tipo (`Int64` en este caso) también lo es; existen otras estructuras que *no* son inmutables.

Esto de ser inmutable no es raro: los números son inmutables (sino, uno podría redefinir "2"). 

Todo aquéllo que es mutable puede ser redefinido. Por ejemplo:

In [ ]:
v = [1,2]

v[2] = 3

v

In [ ]:
push!(v,6)

Los vectores (o cualquier tipo de arreglos) son mutables. Las tuplas, en cambio, **no son** mutables; esto permite un manejo más eficiente del objeto en memoria.

In [ ]:
tup = (1,2.3)

In [ ]:
tup[1] = 2

In [ ]:
tup[1]

El definir una estructura parametrizada hace que, por default, si los parámetros **no** cumplen la signatura del tipo, haya un error:

In [ ]:
Vector2D(1, 2.3)

Noten que la actual definición de `Vector2D{T}` (parametrizada) es *demasiado* flexible, esto es, permite definir ciertos tipos que, quizás, no nos interesa considerar.

In [ ]:
Vector2D("Soy", "Luis")

Las funciones, de hecho, también pueden ser parametrizadas:

In [ ]:
import Base.+
+{T}(a::Vector2D{T}, b::Vector2D{T}) = Vector2D{T}(a.x+b.x, a.y+b.y)

Noten arriba que la función, de hecho, está parametrizada, y que el parámetro está relacionado con el parámetros del tipo de los argumentos. 



In [ ]:
Vector2D(1,3) + Vector2D(-1,2)

In [ ]:
Vector2D(1,3) + Vector2D("Soy", "Luis")

In [ ]:
Vector2D(1,3) + Vector2D(3.0,1.1)

Uno, de hecho, podría haber definido la misma operación sin incluir la parte paramétrica; esto tiene consecuencias interesantes. Así, tenemos:

In [ ]:
import Base.-
-(a::Vector2D, b::Vector2D) = Vector2D(a.x-b.x, a.y-b.y)

In [ ]:
Vector2D(1,3) - Vector2D(-1,2)

In [ ]:
Vector2D(1,3) - Vector2D(3.0,1.1)

In [ ]:
Vector2D(1,3) - Vector2D("Soy", "Luis")

Gracias a que *no* impusimos que `a` y `b` sean *ambos* `Vector2D{T}`, julia puede restarlos.

### Promotion and convertion

La pregunta es, cómo conseguir que *no* nos de un error la instrucción `Vector2D(1, 3.2)`. La respuesta tiene que ver con definir reglas de conversión y promoción. (En este caso concreto, como veremos abajo, una regla de promoción es suficiente.)

Convertir entre tipos (donde esto tenga sentido) se logra usando `convert`:

In [ ]:
convert(Float64, 1//2)

In [ ]:
convert(Array{Float64,1}, [1//2, 1//3])

In [ ]:
x = convert(Rational{Int128}, 0.5)

In [ ]:
typeof(x)

La operación de conversión debe tener sentido; por ejemplo, ¿qué esperamos que nos de `convert(Int64, 0.6)`?

In [ ]:
convert(Int64, 0.6)

In [ ]:
methods(convert)

In [ ]:
convert(Int, true)

In [ ]:
@which convert(Int, true)

También, existe la promoción de varios objetos a un tipo común:

In [ ]:
tup = promote(1, 1//2, BigInt(2))

In [ ]:
typeof(tup)

En julia, hay un árbol jerárquico en la organización de los tipos. Para entender esto veamos dos casos:

In [ ]:
? Int64

In [ ]:
? Rational

In [ ]:
? Signed

Esto muestra que hay ciertos tipos *concretos* (como `Int64`, `Float64`) y *otros* que son abstractos; los segundos en algún sentido agrupan varios tipos distintos.

La instrucción `super` permite ver qué tipo está directamente arriba en la estructura del árbol:

In [ ]:
super(Float64)

In [ ]:
super(AbstractFloat)

In [ ]:
super(Real)

... y la instrucción subtypes, qué está por debajo:

In [ ]:
subtypes(Real)

In [ ]:
subtypes(Number)

La estructura de árbol tiene un tope *por arriba*, `Any`:

In [ ]:
super(Number)

In [ ]:
super(Any)

In [ ]:
subtypes(Any)

Vale la pena notar que existe, entre varias cosas, el tipo `Void` (antes `Nothing`)

In [ ]:
? Void

Entonces, volviendo al punto de cómo hacer para que `Vector2D{1, 3.2}` funcione, podemos definir un nuevo método, para la creación del tipo `Vector2D`, que involucre una promoción de tipos, de la siguiente manera:

In [ ]:
Vector2D(a, b) = Vector2D(promote(a,b)...)

In [ ]:
methods(Vector2D)

In [ ]:
Vector2D(1, 3.2)

Claro que, esto sólo tendrá sentido cuando la promoción tiene sentido; cuando no la tiene, arrojará un error:

In [ ]:
Vector2D("Luis", 3.2)

Por último, uno puede restringir el tipo de parámetros de una estructura, usando la notación ` T <: R` que significa que el parámetro `T` es subtipo de `R`. Un ejemplo es `T <: Real`.

In [ ]:
workspace()

In [ ]:
type Vector2D{T<:Real}
    x :: T
    y :: T
end

Agregamos la promoción de los tipos:

In [ ]:
Vector2D(a, b) = Vector2D(promote(a,b)...)

In [ ]:
Vector2D(1, 3.2)

In [ ]:
Vector2D("Soy", "Luis")

La notación `T <: R` de hecho puede ser utilizada para saber si algo es subtipo de otra cosa:

In [ ]:
Int64 <: Real

In [ ]:
Int64 <: Void

In [ ]:
Void <: Int64

Las dos últimas instrucciones muestran que `Void` e `Int64` pertenecen a dos ramas distintas del árbol jerárquico de tipos; en otras palabras, el tipo superior común entre `Int64` y `Void` es `Any`:

In [ ]:
promote_type(Int64, Void)

## Referencias

Parte de este notebook está basado en [Invitation to julia](https://github.com/dpsanders/invitation_to_julia), de [David P. Sanders](https://github.com/dpsanders).